In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 表格化工作流程：宽度和深度管道

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabular_workflows/wide_and_deep_on_vertex_pipelines.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabular_workflows/wide_and_deep_on_vertex_pipelines.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/tabular_workflows/wide_and_deep_on_vertex_pipelines.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本笔记本展示了如何使用Vertex AI Tabular Workflows运行Wide & Deep算法。

了解更多关于[Wide & Deep的Tabular Workflow](https://cloud.google.com/vertex-ai/docs/tabular-data/tabular-workflows/wide-and-deep)。

### 目标

在本教程中，您将学习如何使用Vertex AI Wide & Deep表格工作流创建两个分类模型。每个工作流都是[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)的托管实例。

本教程使用以下Google Cloud ML服务和资源：

- Vertex AI训练
- Vertex管道
- 云存储

执行的步骤包括：

- 创建一个Wide & Deep CustomJob。如果您知道要用于训练的超参数，这是最佳选项。
- 创建一个Wide & Deep HyperparameterTuningJob。这使您可以获得适合您数据集的最佳超参数集。

训练后，每个管道都会返回一个链接到Vertex Model UI。您可以使用UI部署模型，获得在线预测或运行批量预测。

数据集

您将使用的数据集是[银行营销](https://archive.ics.uci.edu/ml/datasets/bank+marketing)。
这些数据是来自葡萄牙银行机构的直接营销活动（电话营销）数据。二元分类目标是预测客户是否会订阅定期存款。对于这个笔记本，我们随机选择了原始数据集中90%的行，并将它们保存在一个名为train.csv的文件中，该文件托管在云存储中。要下载该文件，请点击[这里](https://storage.googleapis.com/cloud-samples-data-us-central1/vertex-ai/tabular-workflows/datasets/bank-marketing/train.csv)。

### 成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预计使用量生成成本估算。

## 安装

安装以下所需的软件包以执行这个笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 "google-cloud-pipeline-components<2.0"

只有Colab:取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置你的Google Cloud项目

**以下步骤是必需的，无论你使用什么笔记本环境。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当你第一次创建一个账户时，你会获得$300的免费信用额度用于支付计算/存储成本。

2. [确保为你的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用以下API：Vertex AI API、Cloud Resource Manager API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,cloudresourcemanager.googleapis.com)。

4. 如果你是在本地运行这个笔记本，你需要安装[Cloud SDK](https://cloud.google.com/sdk)。

将您的项目ID设定为以下值

**如果您不知道您的项目ID**，请尝试以下操作：
- 运行 `gcloud config list`。
- 运行 `gcloud projects list`。
- 查看支持页面：[定位项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 确认您的 Google Cloud 帐户

根据您的 Jupyter 环境，您可能需要手动进行身份验证。请按照以下相关指示进行操作。

**1. Vertex AI Workbench**
* 不需要进行任何操作，因为您已经验证通过。

**2. 本地 JupyterLab 实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples给您的服务账户授予云存储权限。

### 创建一个云存储桶

创建一个存储桶来存储中间产物，例如数据集。

当您使用Cloud SDK提交一个训练作业时，您需要将包含训练代码的Python包上传至一个云存储桶。Vertex AI 会从这个包中运行代码。在本教程中，Vertex AI 还会将训练作业生成的训练模型保存在同一个存储桶中。使用这个模型产物，您可以创建Vertex AI 模型资源并用于预测。

In [ ]:
BUCKET_URI = "gs://your-bucket-name-unique"  # @param {type:"string"}

只有在你的存储桶不存在的情况下才能运行以下单元格来创建你的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

## 有关服务账户和权限的说明

**默认情况下不需要配置**，如果遇到任何与权限相关的问题，请确保服务账户具有[TabNet、Wide & Deep 以及 Prophet 文档](https://cloud.google.com/vertex-ai/docs/tabular-data/tabular-workflows/service-accounts#fte-workflow)中列出的所需角色。

导入库并定义常量

In [ ]:
# Import required modules
import os
import uuid
from typing import Any, Dict, List

from google.cloud import aiplatform, storage
from google_cloud_pipeline_components.experimental.automl.tabular import \
    utils as automl_tabular_utils

初始化Python的Vertex AI SDK

为您的项目初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

### 定义辅助函数
定义以下辅助函数：

- `get_model_artifacts_path`：从任务详情中获取模型文件路径。
- `get_model_uri`：从任务详情中获取模型 URI。
- `get_bucket_name_and_path`：获取存储桶名称和路径。
- `download_from_gcs`：从存储桶下载内容。
- `write_to_gcs`：将内容上传到存储桶。
- `get_task_detail`：通过任务名称获取任务详情。
- `get_model_name`：从流水线作业 ID 获取模型名称。
- `get_evaluation_metrics`：从流水线任务详情中获取评估指标。

In [ ]:
def get_model_artifacts_path(task_details: List[Dict[str, Any]], task_name: str) -> str:
    task = get_task_detail(task_details, task_name)
    return task.outputs["unmanaged_container_model"].artifacts[0].uri


def get_model_uri(task_details: List[Dict[str, Any]]) -> str:
    task = get_task_detail(task_details, "model-upload")
    # in format https://<location>-aiplatform.googleapis.com/v1/projects/<project_number>/locations/<location>/models/<model_id>
    model_id = task.outputs["model"].artifacts[0].uri.split("/")[-1]
    return f"https://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{model_id}?project={PROJECT_ID}"


def get_bucket_name_and_path(uri: str) -> str:
    no_prefix_uri = uri[len("gs://") :]
    splits = no_prefix_uri.split("/")
    return splits[0], "/".join(splits[1:])


def download_from_gcs(uri: str) -> str:
    bucket_name, path = get_bucket_name_and_path(uri)
    storage_client = storage.Client(project=PROJECT_ID)
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(path)
    return blob.download_as_string()


def write_to_gcs(uri: str, content: str):
    bucket_name, path = get_bucket_name_and_path(uri)
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(path)
    blob.upload_from_string(content)


def get_task_detail(
    task_details: List[Dict[str, Any]], task_name: str
) -> List[Dict[str, Any]]:
    for task_detail in task_details:
        if task_detail.task_name == task_name:
            return task_detail


def get_model_name(job_id: str) -> str:
    pipeline_task_details = aiplatform.PipelineJob.get(
        job_id
    ).gca_resource.job_detail.task_details
    upload_task_details = get_task_detail(pipeline_task_details, "model-upload")
    return upload_task_details.outputs["model"].artifacts[0].metadata["resourceName"]


def get_evaluation_metrics(
    task_details: List[Dict[str, Any]],
) -> str:
    ensemble_task = get_task_detail(task_details, "model-evaluation")
    return download_from_gcs(
        ensemble_task.outputs["evaluation_metrics"].artifacts[0].uri
    )

定义训练规范

### 配置数据集

您可以定义以下任一参数：

- `data_source_csv_filenames`：CSV 数据源。
- `data_source_bigquery_table_path`：BigQuery 数据源。

***注意***：请注意，数据集的存储位置必须与用于启动训练流水线的服务位置（即 `REGION`）相同。

In [ ]:
data_source_csv_filenames = "gs://cloud-samples-data-us-central1/vertex-ai/tabular-workflows/datasets/bank-marketing/train.csv"
data_source_bigquery_table_path = (
    None  # @param {type:"string"}, format: bq://bq_project.bq_dataset.bq_table
)

### 配置特征转换

可通过特征转换引擎（FTE）特定配置指定转换。 FTE支持基于TensorFlow的行级和基于BigQuery的数据集级转换。

* 基于TensorFlow的行级转换：
  * 完全自动转换：FTE根据数据统计信息自动为每个输入列配置一组内置转换。 可通过训练管道中的`tf_auto_transform_features`设置。
  * 全部指定的转换：对输入列上的所有转换明确指定为FTE的内置转换。 还支持在单个列上链接多个转换。 可将这些转换保存为JSON配置文件，并通过训练管道的`tf_transformations_path`参数指定。
  * 自定义转换：自定义的，自定义的转换函数，您可以定义和导入自己的转换函数，并与其他FTE的内置转换一起使用。 您可以将自定义转换指定为JSON对象的数组，并通过训练管道的`tf_custom_transformation_definitions`参数传递。

* 基于BigQuery的数据集级转换：
  * 全部指定的转换：对输入列上的所有转换明确指定为FTE的内置转换。 这些转换可以通过训练管道的`dataset_level_transformations`参数指定为JSON对象的数组。
  * 自定义转换：自定义的，自定义的转换函数，您可以定义和导入自己的转换函数，并与其他FTE的内置转换一起使用。 您可以将自定义转换指定为JSON对象的数组，并通过训练管道的`dataset_level_custom_transformation_definitions`参数传递。

在下面，通过将输入特征列表指定为传递给训练管道的`tf_auto_transform_features`参数来配置完全自动转换。

有关支持的特征转换配置和示例的完整列表，请访问[此处]（https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.31/google_cloud_pipeline_components.experimental.automl.tabular.html#google_cloud_pipeline_components.experimental.automl.tabular.FeatureTransformEngineOp）。

In [ ]:
auto_transform_features = [
    "age",
    "job",
    "marital",
    "education",
    "default",
    "balance",
    "housing",
    "loan",
    "contact",
    "day",
    "month",
    "duration",
    "campaign",
    "pdays",
    "previous",
    "poutcome",
]

### 配置特征选择

除了转换之外，您还可以通过Feature Transform Engine应用特征选择，仅使用由支持的算法评估的高排名特征。如果启用了特征选择，它将在数据集级转换之后立即应用，并排除任何未被选中的特征。

要启用它，您需要将`run_feature_selection`设置为True。

要配置要使用的算法和要选择的特征数量，您需要同时配置`feature_selection_algorithm`和`max_selected_features`参数。

有关支持的特征选择算法和配置的完整列表，请访问[这里](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.31/google_cloud_pipeline_components.experimental.automl.tabular.html#google_cloud_pipeline_components.experimental.automl.tabular.FeatureTransformEngineOp)。

In [ ]:
RUN_FEATURE_SELECTION = True  # @param {type:"boolean"}

FEATURE_SELECTION_ALGORITHM = "AMI"  # @param {type:"string"}

MAX_SELECTED_FEATURES = 10  # @param {type:"integer"}

### 设置训练配置

您需要定义以下内容：

- `target_column`: 目标列的名称。
- `prediction_type`: 模型要产生的预测类型。'classification' 或 'regression'。
- `predefined_split_key`: 预定义拆分列的名称。
- `timestamp_split_key`: 时间戳拆分列的名称。
- `stratified_split_key`: 分层拆分列的名称。
- `training_fraction`: 训练数据的比例。
- `validation_fraction`: 验证数据的比例。
- `test_fraction`: 测试数据的比例。
- `weight_column`: 权重列的名称。
- `run_evaluation`: 是否在训练期间运行评估步骤。

In [ ]:
run_evaluation = True  # @param {type:"boolean"}
prediction_type = "classification"
target_column = "deposit"

# Fraction split
training_fraction = 0.8
validation_fraction = 0.1
test_fraction = 0.1

timestamp_split_key = None  # timestamp column name when using timestamp split
stratified_split_key = None  # target column name when using stratified split
training_fraction = 0.8
validation_fraction = 0.1
test_fraction = 0.1

predefined_split_key = None
if predefined_split_key:
    training_fraction = None
    validation_fraction = None
    test_fraction = None

weight_column = None

## VPC相关配置

您定义如下：

- `dataflow_subnetwork`：Dataflow的完全限定子网络名称，如果为空将使用默认子网络。示例：https://cloud.google.com/dataflow/docs/guides/specifying-networks#example_network_and_subnetwork_specifications
- `dataflow_use_public_ips`：指定Dataflow工作节点是否使用公共IP地址。

如果需要使用自定义Dataflow子网络，可以通过`dataflow_subnetwork`参数进行设置。要求如下：
1. `dataflow_subnetwork`必须是完全限定的子网名称。[[参考链接](https://cloud.google.com/dataflow/docs/guides/specifying-networks#example_network_and_subnetwork_specifications)]
1. 下列服务账户必须在指定的Dataflow子网络上分配[Compute Network User角色](https://cloud.google.com/compute/docs/access/iam#compute.networkUser)：  
    1. Compute Engine默认服务账户：PROJECT_NUMBER-compute@developer.gserviceaccount.com  
    2. Dataflow服务账户：service-PROJECT_NUMBER@dataflow-service-producer-prod.iam.gserviceaccount.com

如果您的项目已启用VPC-SC，请确保：

1. VPC-SC中使用的Dataflow子网络已正确配置。
   [[参考链接](https://cloud.google.com/dataflow/docs/guides/routes-firewall)]
1. `dataflow_use_public_ips`设置为False。

In [ ]:
dataflow_subnetwork = ""  # @param {type:"string"}
dataflow_use_public_ips = True  # @param {type:"boolean"}

## 自定义 Wide & Deep CustomJob 配置并创建流水线

如果您确切地知道要用于模型训练的超参数数值，这是最佳选择。它比 HyperparameterTuningJob 使用更少的训练资源。

在下面的示例中，您可以配置以下内容：

- `root_dir`: 用于流水线组件的根 GCS 目录。
- `worker_pool_specs_override`: 用于覆盖训练和评估工作池规范的字典。该字典应该是[这种格式](https://github.com/googleapis/googleapis/blob/4e836c7c257e3e20b1de14d470993a2b1f4736a8/google/cloud/aiplatform/v1beta1/custom_job.proto#L172)。
- `learning_rate`: 线性优化器使用的学习率。
- `dnn_learning_rate`: 训练模型深部分使用的学习率。
- `max_steps`: 运行训练器的步数。
- `max_train_secs`: 运行训练器的时间量（以秒为单位）。

可以在[这里](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.23/google_cloud_pipeline_components.experimental.automl.tabular.html#google_cloud_pipeline_components.experimental.automl.tabular.utils.get_tabnet_trainer_pipeline_and_parameters)找到完整的流水线输入和模型超参数列表。

In [ ]:
pipeline_job_root_dir = os.path.join(BUCKET_URI, "wide_and_deep_custom_job")

# max_steps and/or max_train_secs must be set. If both are
# specified, training will stop after either condition is met.
# By default, max_train_secs is set to -1.

max_steps = 1000
max_train_secs = -1

learning_rate = 0.01
dnn_learning_rate = 0.01

worker_pool_specs_override = [
    {"machine_spec": {"machine_type": "c2-standard-16"}}  # Override for TF chief node
]

# To test GPU training, the worker_pool_specs_override can be specified like this.
# worker_pool_specs_override =  [
#     {"machine_spec": {
#       'machine_type': "n1-highmem-32",
#       "accelerator_type": "NVIDIA_TESLA_V100",
#       "accelerator_count": 2
#       }
#     }
#   ]

# If your system does not use Python, you can save the JSON file (`template_path`),
# and use another programming language to submit the pipeline.
(
    template_path,
    parameter_values,
) = automl_tabular_utils.get_wide_and_deep_trainer_pipeline_and_parameters(
    project=PROJECT_ID,
    location=REGION,
    root_dir=pipeline_job_root_dir,
    max_steps=max_steps,
    max_train_secs=max_train_secs,
    learning_rate=learning_rate,
    dnn_learning_rate=dnn_learning_rate,
    target_column=target_column,
    prediction_type=prediction_type,
    tf_auto_transform_features=auto_transform_features,
    run_feature_selection=RUN_FEATURE_SELECTION,
    feature_selection_algorithm=FEATURE_SELECTION_ALGORITHM,
    max_selected_features=MAX_SELECTED_FEATURES,
    training_fraction=training_fraction,
    validation_fraction=validation_fraction,
    test_fraction=test_fraction,
    data_source_csv_filenames=data_source_csv_filenames,
    data_source_bigquery_table_path=data_source_bigquery_table_path,
    worker_pool_specs_override=worker_pool_specs_override,
    dataflow_use_public_ips=dataflow_use_public_ips,
    dataflow_subnetwork=dataflow_subnetwork,
    run_evaluation=run_evaluation,
)

pipeline_job_id = f"wide-and-deep-{uuid.uuid4()}"
# More info on parameters PipelineJob accepts:
# https://cloud.google.com/vertex-ai/docs/pipelines/run-pipeline#create_a_pipeline_run
pipeline_job = aiplatform.PipelineJob(
    display_name=pipeline_job_id,
    template_path=template_path,
    job_id=pipeline_job_id,
    pipeline_root=pipeline_job_root_dir,
    parameter_values=parameter_values,
    enable_caching=False,
)

pipeline_job.run()

### 前往顶点模型 UI
你可以通过下方的链接部署模型、进行在线预测或批量预测。

In [ ]:
wide_and_deep_trainer_pipeline_task_details = aiplatform.PipelineJob.get(
    pipeline_job_id
).gca_resource.job_detail.task_details
CUSTOM_JOB_MODEL = get_model_name(pipeline_job_id)
print("model uri:", get_model_uri(wide_and_deep_trainer_pipeline_task_details))
print(
    "model artifacts:",
    get_model_artifacts_path(
        wide_and_deep_trainer_pipeline_task_details, "wide-and-deep-trainer"
    ),
)

## 自定义广泛和深度超参数调整作业配置并创建管道

为了获得最佳的超参数组合，建议运行一个HyperparameterTuningJob。

可以调整的超参数在可选的`study_spec_parameters_override`参数中设置。我们提供了一个称为`get_wide_and_deep_study_spec_parameters_override`的辅助函数来获取这些超参数。该函数返回一个超参数和范围的列表。`study_spec_parameters_override`可以为空，或者可以指定一个或多个这些超参数。对于未在`study_spec_parameters_override`中指定的超参数，我们在管道中设置范围。有关可用于调整的超参数的完整列表，请参阅[这里](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.23/google_cloud_pipeline_components.experimental.automl.tabular.html#google_cloud_pipeline_components.experimental.automl.tabular.utils.get_wide_and_deep_trainer_pipeline_and_parameters)。

除了超参数外，HyperparameterTuningJob在下面的示例中还采用以下值：

- `root_dir`：管道组件的根GCS目录。
- `worker_pool_specs_override`：用于覆盖训练和评估工作池规范的字典。字典应该是[此格式](https://github.com/googleapis/googleapis/blob/4e836c7c257e3e20b1de14d470993a2b1f4736a8/google/cloud/aiplatform/v1beta1/custom_job.proto#L172)。
- `study_spec_metric_id`：优化的指标，可能的值：['loss', 'average_loss', 'rmse', 'mae', 'mql', 'accuracy', 'auc', 'precision', 'recall']。
- `study_spec_metric_goal`：指标的优化目标，可能的值："MAXIMIZE"，"MINIMIZE"。
- `max_trial_count`：所需的总试验数。
- `parallel_trial_count`：要并行运行的试验数。
- `max_failed_trial_count`：在失败HyperparameterTuningJob之前所需看到的失败试验数。如果设置为0，Vertex AI将决定在整个作业失败之前必须失败的试验数量。
- `study_spec_algorithm`：为研究指定的搜索算法。其中之一是'ALGORITHM_UNSPECIFIED'，'GRID_SEARCH'或者' RANDOM_SEARCH'。

有关HyperparameterTuningJob参数的完整列表，请参见[这里](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.23/google_cloud_pipeline_components.experimental.automl.tabular.html#google_cloud_pipeline_components.experimental.automl.tabular.utils.get_wide_and_deep_hyperparameter_tuning_job_pipeline_and_parameters)。

可以配置多个试验。管道会返回基于`study_spec_metrics`配置的指标的最佳试验。在下面的示例中，我们返回具有最低损失值的试验。

In [ ]:
pipeline_job_root_dir = os.path.join(
    BUCKET_URI, "wide_and_deep_hyperparameter_tuning_job"
)

worker_pool_specs_override = [
    {"machine_spec": {"machine_type": "c2-standard-16"}}  # Override for TF chief node
]

# To test GPU training, the worker_pool_specs_override can be specified like this.
# worker_pool_specs_override =  [
#    {
#       "machine_spec":{
#          "machine_type":"n1-highmem-32",
#          "accelerator_type":"NVIDIA_TESLA_V100",
#          "accelerator_count":2
#       }
#    }
# ]

study_spec_metric_id = "loss"
study_spec_metric_goal = "MINIMIZE"

# max_steps and/or max_train_secs must be set. If both are
# specified, training will stop after either condition is met.
# By default, max_train_secs is set to -1 and max_steps is set to
# an appropriate range given dataset_size and training budget.
study_spec_parameters_override = (
    automl_tabular_utils.get_wide_and_deep_study_spec_parameters_override()
)

# If your system does not use Python, you can save the JSON file (`template_path`),
# and use another programming language to submit the pipeline.
(
    template_path,
    parameter_values,
) = automl_tabular_utils.get_wide_and_deep_hyperparameter_tuning_job_pipeline_and_parameters(
    project=PROJECT_ID,
    location=REGION,
    root_dir=pipeline_job_root_dir,
    target_column=target_column,
    prediction_type=prediction_type,
    tf_auto_transform_features=auto_transform_features,
    run_feature_selection=RUN_FEATURE_SELECTION,
    feature_selection_algorithm=FEATURE_SELECTION_ALGORITHM,
    max_selected_features=MAX_SELECTED_FEATURES,
    training_fraction=training_fraction,
    validation_fraction=validation_fraction,
    test_fraction=test_fraction,
    data_source_csv_filenames=data_source_csv_filenames,
    data_source_bigquery_table_path=data_source_bigquery_table_path,
    study_spec_metric_id=study_spec_metric_id,
    study_spec_metric_goal=study_spec_metric_goal,
    study_spec_parameters_override=study_spec_parameters_override,
    max_trial_count=1,
    parallel_trial_count=1,
    max_failed_trial_count=0,
    worker_pool_specs_override=worker_pool_specs_override,
    dataflow_use_public_ips=dataflow_use_public_ips,
    dataflow_subnetwork=dataflow_subnetwork,
    run_evaluation=True,
)

pipeline_job_id = f"wide-and-deep-hpt-{uuid.uuid4()}"
# More info on parameters PipelineJob accepts:
# https://cloud.google.com/vertex-ai/docs/pipelines/run-pipeline#create_a_pipeline_run
pipeline_job = aiplatform.PipelineJob(
    display_name=pipeline_job_id,
    template_path=template_path,
    job_id=pipeline_job_id,
    pipeline_root=pipeline_job_root_dir,
    parameter_values=parameter_values,
    enable_caching=False,
)

pipeline_job.run()

### 前往顶点模型 UI
通过下面的链接，您可以部署模型并进行在线预测测试，或者运行批量预测。

In [ ]:
wide_and_deep_hpt_pipeline_task_details = aiplatform.PipelineJob.get(
    pipeline_job_id
).gca_resource.job_detail.task_details
HPT_JOB_MODEL = get_model_name(pipeline_job_id)
print("model uri:", get_model_uri(wide_and_deep_hpt_pipeline_task_details))
print(
    "model artifacts:",
    get_model_artifacts_path(
        wide_and_deep_hpt_pipeline_task_details,
        "get-best-hyperparameter-tuning-job-trial",
    ),
)

清理顶点和BigQuery资源

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- Cloud Storage 存储桶
- CustomJob 管道生成的模型
- HyperparameterTuningJob 管道生成的模型

In [ ]:
# Delete model resources
custom_job_model = aiplatform.Model(CUSTOM_JOB_MODEL)
hpt_job_model = aiplatform.Model(HPT_JOB_MODEL)
custom_job_model.delete()
hpt_job_model.delete()

# Delete bucket
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI